# String predictor

Tablog encodes column name strings in headers and we want to compress them at least a bit -- the strings should not be the dominant part of the dataset, but still saving few bytes is always good.

This file contains experiments that should help determining an appropriate compression algorithm.

Goals for this part are (in order of importance):
1. fast and light
2. simple-ish to implement
3. compresses well enough to make me happy

In [1]:
import os
import re
import collections
import math
import string
import heapq
import itertools
import string
import gzip
import dataclasses

In [2]:
import requests
from tqdm.notebook import tqdm, trange

## Manual test data
These are just a couple of strings that I feel will be representative of the intended use of Tablog.
Used mostly to get an intuitive feel of how the compression behaves.

In [3]:
manual_test_data = [
    "deadbeef", "hello_world", "voltage", "current", "left", "column58", "left_motor_speed", "kP", "right_motor_kD",
    "POWER!", "timestamp", "AverageTicks", "temperature", "MCU-power", "alpha", "beta", "omega", "velocityX",
    "+-!@#$%^&*()[]{}", ""
]

## English words

In [4]:
def google_books_1grams_words():
    ret = collections.Counter()
    good_string_re = re.compile(r"^([a-zA-Z]*)\t[^\t]*\t([0-9]*)")
    for letter in tqdm(string.ascii_lowercase):
        r = requests.get(f"http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-1gram-20120701-{letter}.gz", stream=True)
        file_size = int(r.headers.get('Content-Length', 0))
        with tqdm.wrapattr(r.raw, "read", total=file_size, desc=letter) as compressed_response_file:
            with gzip.open(compressed_response_file, "rt") as response_file:
                for line in response_file:
                    if match := good_string_re.match(line):
                        ret[match[1]] += int(match[2])
                        
    return ret

In [5]:
english_words = google_books_1grams_words()

  0%|          | 0/26 [00:00<?, ?it/s]

a:   0%|          | 0/343487895 [00:00<?, ?it/s]

b:   0%|          | 0/247964820 [00:00<?, ?it/s]

c:   0%|          | 0/393712861 [00:00<?, ?it/s]

d:   0%|          | 0/239389478 [00:00<?, ?it/s]

...

v:   0%|          | 0/108848534 [00:00<?, ?it/s]

w:   0%|          | 0/118835349 [00:00<?, ?it/s]

x:   0%|          | 0/14649720 [00:00<?, ?it/s]

y:   0%|          | 0/26595711 [00:00<?, ?it/s]

z:   0%|          | 0/27324007 [00:00<?, ?it/s]

In [32]:
english_words.most_common(30)  # Just a peek

[('the', 23688414489),
 ('of', 15342397280),
 ('and', 11021132912),
 ('to', 9494905988),
 ('in', 7611765281),
 ('a', 7083003595),
 ('is', 4139526351),
 ('that', 3870260345),
 ('for', 3021925527),
 ('The', 2763139452),
 ('was', 2737725870),
 ('as', 2626386982),
 ('with', 2504225400),
 ('be', 2392576396),
 ('by', 2249978492),
 ('not', 2208497726),
 ('it', 2204134503),
 ('on', 2143313303),
 ('I', 1873234887),
 ('are', 1826889845),
 ('or', 1805149769),
 ('his', 1672173489),
 ('from', 1651527506),
 ('at', 1562321315),
 ('which', 1557215562),
 ('he', 1529423270),
 ('this', 1423199366),
 ('have', 1372997478),
 ('had', 1298386780),
 ('an', 1266190915)]

In [7]:
def ngrams(data, n):
    """Generate a counter with all length n substrings of strings in the input counter"""
    counts = collections.Counter()
    for s, count in data.items():
        for i in range(len(s) - n + 1):
            counts[s[i:i + n]] += count
    return counts

In [8]:
def ngrams_up_to(data, max_n):
    """Generate a list of counters with all length 1-maxn substrings of strings in the input counter"""
    ret = collections.Counter()
    for s, count in tqdm(data.items(), desc=f"Collecting (1..{max_n})-grams"):
        for j in range(1, min(len(s), max_n) + 1):
            for i in range(len(s) - j + 1):
                ret[s[i:i + j]] += count
    return ret

In [9]:
english_words_ngrams = ngrams_up_to(english_words, 4)

## Dictionary extras
Padding the dictionary with ngrams by stuff that we expect to appear in the inputs, but which is not contained in the english words dataset.

The weights of different symbols are manually estimated and may be significantly non-optimal.

### Numbers

In [10]:
def generate_numbers_ngrams(p, max_value, max_ngram_length):
    ret = collections.Counter()
    for i in range(max_value):
        weight = p * (1 - p)**i
        s = str(i)
        for n in range(1, max_ngram_length + 1):
            for k in range(len(s) - n + 1):
                ret[s[k:k + n]] += weight
    for i in range(int(math.log2(max_value))):
        weight =  p * (1 - p)**i
        s = str(2**i)
        for n in range(1, max_ngram_length + 1):
            for k in range(len(s) - n + 1):
                ret[s[k:k + n]] += weight
    for i in range(int(math.log10(max_value))):
        weight = 0.5 * p * (1 - p)**i
        s = str(10**i)
        for n in range(1, max_ngram_length + 1):
            for k in range(len(s) - n + 1):
                ret[s[k:k + n]] += weight
    return ret

In [11]:
number_ngrams = generate_numbers_ngrams(p=0.25, max_value=10000, max_ngram_length=5)

### Symbols

In [12]:
symbols_ngrams = collections.Counter({
    "_": 1, " ": .5, "-": .2, "/": .1, ".": .1,
    ",": .02, ", ": .2, ". ": .2, ",\n": .05, ";\n": .1, "; ": .05,
    "\r\n": .05,
    "http": .05, "://": 0.1, ".com": .07, ".co": .005, "&": 0.05, "?": 0.025,
    "@": .05
})

### Capitals

In [14]:
def map_ngrams(ngrams, func):
    """ Map all keys of ngrams """
    ret = collections.Counter()
    for ngram, count in tqdm(ngrams.items()):
        ret[func(ngram)] += count
    return ret

In [15]:
caps_english_words_ngrams = map_ngrams(english_words_ngrams, lambda x: x.upper())

  0%|          | 0/832900 [00:00<?, ?it/s]

## Merging ngrams

In [16]:
def merge_ngrams(*weighted_ngrams):
    """ Merge tuples of (weight, ngrams) to a single ngram block.
    Weights within each ngram block are normalized first, so that they sum up to 1. """
    ret = collections.Counter()
    for weight, ngrams in tqdm(weighted_ngrams):
        factor = weight / sum(ngrams.values())

        for ngram, count in tqdm(ngrams.items()):
            ret[ngram] += factor * count
    return ret

In [17]:
merged_ngrams = merge_ngrams(
    (1, english_words_ngrams),
    (0.02, caps_english_words_ngrams),
    (0.02, number_ngrams),
    (0.05, symbols_ngrams)
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/832900 [00:00<?, ?it/s]

  0%|          | 0/270290 [00:00<?, ?it/s]

  0%|          | 0/10110 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [18]:
def filter_ngrams(input_ngrams, n):
    return collections.Counter({
        ngram: count
        for (ngram, count)
        in heapq.nlargest(
            n,
            input_ngrams.items(),
            key=lambda item: len(item[0]) * item[1]
        )
    })

In [19]:
def add_prefixes(filtered_ngrams, all_ngrams, nonrepreseneted_weight_fraction=0):
    """Extends the filtered trie to contain all prefixes. If a prefix is not represeneted in the
    input ngrams, its weight is calculated by multiplying the parent ngram weight by `nonrepreseneted_weight_fraction`.
    This is mainly useful to artificially  boost their zero probabilities to a reasonable value and avoid very
    long huffman codes (which would later be problematic during encoding)."""
    ret = filtered_ngrams.copy()
    for ngram in filtered_ngrams:
        for i in range(len(ngram)):
            substr = ngram[:i+1]
            if substr not in ret:
                if substr not in all_ngrams:
                    ret[substr] = nonrepreseneted_weight_fraction * all_ngrams[ngram]
                else:
                    ret[substr] = all_ngrams[substr]
                
    print(f"Added {len(ret) - len(filtered_ngrams)} ngrams")
    return ret

At this point we need to select which ngrams from the merged ones will be used for the final encoding.
This is done in a very simple way (function `filter_ngrams`), then the ngrams are padded using `add_prefixes` so that the final trie has an encoding for all inner nodes, which significantly simplifies the encoding.

The parameters for these two functions were selected manually by trial and error.
The conditions are:
- Number of final ngrams is greater or equal than the spread between first items in the trie (not explicit at this point)
  - This is done to allow losslessly overlaping the lookup of the first step of the trie by direct lookup with flattened storage of the rest of the trie (details later)
- Longest encoded value is shorter than 16 bit

In [20]:
filtered_merged_ngrams = add_prefixes(
    filter_ngrams(merged_ngrams, n=82),
    merged_ngrams,
    nonrepreseneted_weight_fraction=0.01
)

Added 9 ngrams


In [21]:
def minmax(iterable):
    """Calculate minimum and maximum of an iterable in a single pass"""
    it = iter(iterable)
    try:
        minimum = next(it)
    except StopIteration:
        return (None, None)
    maximum = minimum
    
    for v in it:
        if v < minimum:
            minimum = v
        if v > maximum:
            maximum = v
            
    return (minimum, maximum)

## Variable length Huffman coding

In [22]:
def huffman_tree(symbols):
    """Build a huffman tree from a dict of symbol: probability. """
    
    heap = [(x[1], True, x[0]) for x in symbols.items()]
    # Heap elements look like: probability, leaf flag, (leaf data | children).
    heapq.heapify(heap)
    
    while len(heap) > 1:
        first = heapq.heappop(heap)
        second = heap[0]
        
        if first > second:
            (first, second) = (second, first)
        
        new = (first[0] + second[0], False, (first, second))
        
        heapq.heapreplace(heap, new)
        
    encoding = {}
        
    def flatten_tree(element, current_encoding=""):
        if element[1]:  # is leaf
            encoding[element[2]] = current_encoding
        else:
            flatten_tree(element[2][0], current_encoding + "0")
            flatten_tree(element[2][1], current_encoding + "1")
    
    flatten_tree(heap[0])
    
    return encoding

In [23]:
encoding = huffman_tree(filtered_merged_ngrams)
for tup, enc in sorted(encoding.items(), key=lambda x: (filtered_merged_ngrams[x[0]]), reverse=True):
    print(f"{tup!r} -> {enc} ({len(enc)}b)")

'e' -> 1100 (4b)
't' -> 0111 (4b)
'a' -> 0100 (4b)
'o' -> 0010 (4b)
'i' -> 0000 (4b)
'n' -> 11111 (5b)
's' -> 11100 (5b)
'r' -> 11011 (5b)
'h' -> 10100 (5b)
'_' -> 10011 (5b)
'l' -> 01011 (5b)
'd' -> 00011 (5b)
'c' -> 111010 (6b)
'u' -> 101101 (6b)
'th' -> 101010 (6b)
' ' -> 100101 (6b)
'm' -> 100100 (6b)
'he' -> 100010 (6b)
'f' -> 100000 (6b)
'p' -> 010100 (6b)
'in' -> 001100 (6b)
'g' -> 000100 (6b)
'the' -> 1111010 (7b)
'y' -> 1110111 (7b)
'er' -> 1110110 (7b)
'w' -> 1101010 (7b)
'an' -> 1101001 (7b)
're' -> 1011110 (7b)
'on' -> 1011100 (7b)
'b' -> 1011001 (7b)
'1' -> 1010110 (7b)
'at' -> 1000110 (7b)
'en' -> 0110111 (7b)
'nd' -> 0110011 (7b)
'es' -> 0110010 (7b)
'ti' -> 0110001 (7b)
'v' -> 0101011 (7b)
'or' -> 0101010 (7b)
', ' -> 0011101 (7b)
'-' -> 0011110 (7b)
'. ' -> 0011111 (7b)
'te' -> 0011010 (7b)
'ed' -> 0001010 (7b)
'of' -> 11110111 (8b)
'is' -> 11110110 (8b)
'it' -> 11110011 (8b)
'al' -> 11110010 (8b)
'ar' -> 11110001 (8b)
'nt' -> 11110000 (8b)
'to' -> 11010111 (8b)
'st' -

Make sure that the encoding is actually a prefix code:

In [24]:
for tup1, enc1 in encoding.items():
    for tup2, enc2 in encoding.items():
        if tup1 == tup2:
            continue
        if enc1.startswith(enc2):
            print(f"{b(tup1)} -> {enc1} starts with {b(tup2)} -> {enc2}")
        assert not enc1.startswith(enc2)

### Encoding using a trie

In [25]:
def build_trie(encoding):
    trie = {}
    for tup, enc in sorted(encoding.items(), key=lambda x: x[0]):
        current = trie
        for c in tup[:-1]:
            current = current.setdefault(c, (None, {}))[1]
        if tup[-1] in current:
            assert current[tup[-1]][0] is None
            current[tup[-1]] = (enc, current[tup[-1]][1])
        else:
            current[tup[-1]] = (enc, {})
    
    return trie

In [26]:
def print_trie(trie, indent=""):
    for c, (enc, children) in trie.items():
        print(f"{indent}{c!r}: {enc}")
        print_trie(children, indent + "  ")

In [27]:
trie = build_trie(encoding)
print_trie(trie)

' ': 100101
',': 00110110011
  ' ': 0011101
'-': 0011110
'.': 00110111
  ' ': 0011111
  'c': 001101100100001
    'o': 001101100101
      'm': 110100010
'1': 1010110
':': 00110110010001
  '/': 00110110010010
    '/': 00111000
';': 00110110010011
  '\n': 00111001
'_': 10011
'a': 0100
  'l': 11110010
  'n': 1101001
    'd': 10111111
  'r': 11110001
  's': 10110001
  't': 1000110
    'i': 100001001
      'o': 100001000
'b': 1011001
'c': 111010
  'e': 01100000
  'o': 01101010
'd': 00011
  'e': 10000110
'e': 1100
  'a': 01101000
  'd': 0001010
  'n': 0110111
    't': 110100011
  'r': 1110110
  's': 0110010
'f': 100000
'g': 000100
'h': 10100
  'a': 10111110
  'e': 100010
  'i': 10000111
  't': 0011011000
    't': 001101100100000
      'p': 001101101
'i': 0000
  'c': 01101011
  'n': 001100
    'g': 10001110
  'o': 10110000
    'n': 01101101
  's': 11110110
  't': 11110011
'l': 01011
  'e': 10101111
'm': 100100
  'e': 10001111
'n': 11111
  'd': 0110011
  'e': 01100001
  'g': 11010000
  't': 111

## Trie representation in C++
We flatten the trie into an array, using records containing character for this node, number of children, index of a first child and the resulting encoding for this string.
First level of the trie is accessed using direct array access with offset, the remaining data are interleaved in the gaps remaining behind this representation.
The representation makes sure that the first level access is unique -- character stored in the array corresponds to its location if and only if the item is a first level node.

In [28]:
def pack_trie(trie):
    @dataclasses.dataclass
    class Record:
        c: str
        child_index:int
        child_count: int
        encoding: str
        full_key: str  # Debug only
    
    first_level_bounds = minmax(ord(c) for c in trie.keys())
    
    placement = {}  # subtree key, index
    records = {}
    gaps = []
    
    # Place the fixed first level items, find the usable gaps
    previous = 0
    for c in sorted(trie.keys()):
        index = ord(c) - first_level_bounds[0]
        placement[c] = index
        gap = index - previous
        previous = index + 1
        if gap > 0:
            gaps.append((gap, index - gap))
    gaps.append((float("inf"), first_level_bounds[1] - first_level_bounds[0] + 1)) # There's an infinite gap available at the end
    
    def assert_gaps():
        nonlocal gaps
        assert \
            all(g1[1] + g1[0] < g2[1] for g1, g2 in zip(gaps[:-1], gaps[1:])), \
            "The gaps are ordered and there is at least one non-gap element between them"
    assert_gaps()
    
    
    # Collect the subtrees that need to fill the gaps
    def collect_subtrees(trie, root_key, encoding, subtrees):
        for c, (child_encoding, child_trie) in trie.items():
            collect_subtrees(child_trie, root_key + c, child_encoding, subtrees)
        if root_key: # Skip the top level subtree, because we already handle that one differently
            subtrees.append((len(trie), root_key, encoding, trie.keys()))
    subtrees = []
    collect_subtrees(trie, "", None, subtrees)
    subtrees.sort(reverse=True)
    
    # Place subtrees into the gaps
    for length, key, encoding, children in subtrees:
        assert length == len(children)
        
        possible_gap_indices = sorted((i for i in range(len(gaps)) if gaps[i][0] >= length), key=lambda i: gaps[i][0])
        for selected_gap_index in possible_gap_indices:
            gap_length, gap_start = gaps[selected_gap_index]
            
            if any(ord(child) == first_level_bounds[0] + gap_start + i for i, child in enumerate(children)):
                print("Conflict with top level placement, using different gap")
                continue

            #print(f"placing subtree of key {key!r} (l={length}), into gap {selected_gap_index} ({gap_length}, {gap_start})")

            records[key] = Record(
                key[-1],
                child_index=gap_start if length > 0 else 255,  # Cosmetic value to make the invalid indices more visible
                child_count=length,
                encoding=encoding,
                full_key=key
            )
            
            # Place all subtree children
            for i, child in enumerate(children):
                assert (key + child) not in placement
                #print(f"placing {key + child!r} to {i + gap_start}")
                placement[key + child] = gap_start + i

            # Shorten or delete the gap
            if gap_length == length:
                del gaps[selected_gap_index]
            else:
                gaps[selected_gap_index] = (gap_length - length, gap_start + length)
            assert_gaps()
            
            break
        else:
            raise Exception("Can't pack the trie using the greedy algorithm :-(")
    
    assert len(gaps) == 1, "Only one gap left"
    assert gaps[0][0] == float("inf"), "The remaining gap is the infinite one at the end"
    
    ret = []
    
    for key, position in sorted(placement.items(), key=lambda x: x[1]):
        assert position == len(ret)
        ret.append(records[key])
        
    return ret

In [29]:
packed_trie = pack_trie(trie)

In [30]:
def format_packed_trie(packed_trie):
    def escape_char(c):
        assert len(c) == 1
        if (32 <= ord(c) < 127) and c not in ('\\', '"'):
            return c
        else:
            return f"\\x{ord(c):02x}"
        
    first_level_min = packed_trie[0].c
    first_level_max = max(r.c for r in packed_trie if len(r.full_key) == 1)
        
    print(f"static constexpr char trieFirstLevelMin = '{escape_char(first_level_min)}';")
    print(f"static constexpr char trieFirstLevelMax = '{escape_char(first_level_max)}';")
    print("static constexpr TrieNode trie[] = {");
    for i, r in enumerate(packed_trie):
        c = escape_char(r.c)
        end = "}," if i < (len(packed_trie) - 1) else "}"
        print(f"    {{'{c}', {r.child_index}, {r.child_count}, {len(r.encoding)}, {int(r.encoding, 2)}{end} // {r.full_key!r}")
    print("};");

In [31]:
format_packed_trie(packed_trie)

static constexpr char trieFirstLevelMin = ' ';
static constexpr char trieFirstLevelMax = 'y';
static constexpr TrieNode trie[] = {
    {' ', 255, 0, 6, 37}, // ' '
    {'a', 255, 0, 8, 104}, // 'ea'
    {'d', 255, 0, 7, 10}, // 'ed'
    {'n', 53, 1, 7, 55}, // 'en'
    {'r', 255, 0, 7, 118}, // 'er'
    {'s', 255, 0, 7, 50}, // 'es'
    {'l', 255, 0, 8, 242}, // 'al'
    {'n', 57, 1, 7, 105}, // 'an'
    {'r', 255, 0, 8, 241}, // 'ar'
    {'s', 255, 0, 8, 177}, // 'as'
    {'t', 56, 1, 7, 70}, // 'at'
    {'e', 255, 0, 8, 174}, // 've'
    {',', 90, 1, 11, 435}, // ','
    {'-', 255, 0, 7, 30}, // '-'
    {'.', 23, 2, 8, 55}, // '.'
    {'e', 255, 0, 8, 187}, // 'se'
    {'t', 255, 0, 8, 214}, // 'st'
    {'1', 255, 0, 7, 86}, // '1'
    {'c', 255, 0, 8, 107}, // 'ic'
    {'n', 50, 1, 6, 12}, // 'in'
    {'o', 49, 1, 8, 176}, // 'io'
    {'s', 255, 0, 8, 246}, // 'is'
    {'t', 255, 0, 8, 243}, // 'it'
    {' ', 255, 0, 7, 31}, // '. '
    {'c', 62, 1, 15, 6945}, // '.c'
    {'n', 255,